In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "usage: ipykernel_launcher.py [-h] [--C C] [--max_iter MAX_ITER]\n",
      "ipykernel_launcher.py: error: unrecognized arguments: -f /home/azureuser/.local/share/jupyter/runtime/kernel-d9e4a216-5e4d-4eff-afc5-f5eb71b218fa.json\n"
     ]
    },
    {
     "ename": "SystemExit",
     "evalue": "2",
     "output_type": "error",
     "traceback": [
      "An exception has occurred, use %tb to see the full traceback.\n",
      "\u001b[0;31mSystemExit\u001b[0m\u001b[0;31m:\u001b[0m 2\n"
     ]
    },
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.\n",
      "  warn(\"To exit: use 'exit', 'quit', or Ctrl-D.\", stacklevel=1)\n"
     ]
    }
   ],
   "source": [
    "import os\n",
    "import numpy as np\n",
    "from sklearn.metrics import mean_squared_error\n",
    "import joblib\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.preprocessing import StandardScaler\n",
    "import pandas as pd\n",
    "from azureml.core.run import Run\n",
    "from azureml.data.dataset_factory import TabularDatasetFactory\n",
    "from azureml.core import Dataset\n",
    "import argparse\n",
    "\n",
    "# TOO Create TabularDataset using TabularDataFactory\n",
    "# Data is located at:\n",
    "# \"https://github.com/ohikhatemenG/Machine-Learning-Azure-Capstone/blob/main/diabetes.csv\"\n",
    "web_paths='https://drive.google.com/drive/my-drive/diabetes.csv'\n",
    "\n",
    "ds = Dataset.Tabular.from_delimited_files(path=web_paths)\n",
    "\n",
    "run = Run.get_context()\n",
    "\n",
    "def clean_data(data):\n",
    "    df = data.to_pandas_dataframe().dropna()\n",
    "    df['BMI'] = df['BMI'].replace(0,df['BMI'].mean())\n",
    "    df['BloodPressure'] = df['BloodPressure'].replace(0,df['BloodPressure'].mean())\n",
    "    df['Glucose'] = df['Glucose'].replace(0,df['Glucose'].mean())\n",
    "    df['Insulin'] = df['Insulin'].replace(0,df['Insulin'].mean())\n",
    "    df['SkinThickness'] = df['SkinThickness'].replace(0,df['SkinThickness'].mean())\n",
    "    q = df['Pregnancies'].quantile(0.98)\n",
    "    data_cleaned = df[df['Pregnancies']<q]\n",
    "    q = data_cleaned['BMI'].quantile(0.99)\n",
    "    data_cleaned  = data_cleaned[data_cleaned['BMI']<q]\n",
    "    q = data_cleaned['SkinThickness'].quantile(0.99)\n",
    "    q = data_cleaned['Insulin'].quantile(0.95)\n",
    "    data_cleaned  = data_cleaned[data_cleaned['Insulin']<q]\n",
    "    q = data_cleaned['DiabetesPedigreeFunction'].quantile(0.99)\n",
    "    data_cleaned  = data_cleaned[data_cleaned['DiabetesPedigreeFunction']<q]\n",
    "    q = data_cleaned['Age'].quantile(0.99)\n",
    "    data_cleaned  = data_cleaned[data_cleaned['Age']<q]\n",
    "    \n",
    "    X = df.drop(columns = ['Outcome'])\n",
    "    y = df['Outcome']\n",
    "    scalar = StandardScaler()\n",
    "    X_scaled = scalar.fit_transform(X)\n",
    "    return X_scaled,y\n",
    "\n",
    "\n",
    "def main():\n",
    "    parser = argparse.ArgumentParser()\n",
    "    \n",
    "    parser.add_argument('--C', type=float, default=1.0, help=\"Inverse of regularization strength. Smaller values cause stronger regularization\")\n",
    "    parser.add_argument('--max_iter', type=int, default=100, help=\"Maximum number of iterations to converge\")\n",
    "    \n",
    "    args = parser.parse_args()\n",
    "    \n",
    "    run.log(\"Regularization Strength:\", np.float(args.C))\n",
    "    run.log(\"Max iterations:\", np.int(args.max_iter))\n",
    "    \n",
    "    x,y=clean_data(ds)\n",
    "    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)\n",
    "    \n",
    "    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)\n",
    "    joblib.makedirs('output', exist_ok=True)\n",
    "    joblib.dump(model,'outputs/model.joblib')\n",
    "    \n",
    "    accuracy = model.score(x_test, y_test)\n",
    "    run.log(\"Accuracy\", np.float(accuracy))\n",
    "if __name__ == '__main__':\n",
    "    main()\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3.6 - AzureML",
   "language": "python",
   "name": "python3-azureml"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}